# Publicando Feedback

### Configuração

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Adicionando Feedback a uma Execução

Você pode adicionar feedback a uma Execução existente programaticamente, desde que conheça o run_id. Vamos pegar um run_id da UI do LangSmith e adicioná-lo aqui.

In [ ]:
run_id = ""

Agora, vamos adicionar algum feedback contínuo

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="amostra-continua",
  score=7.0,
  comment="Este é um exemplo de feedback contínuo",
)

E agora, vamos adicionar algum feedback categórico também!

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="amostra-categorica",
  value="não",
  comment="Este é um exemplo de feedback categórico",
)

### Pré-Gerando Run IDs para Feedback

![Generate_run_id](../../images/generate_run_id.png)

Usando LangChain, oferecemos a capacidade de pré-gerar e definir run IDs, antes que seu código seja invocado e o run ID seja gerado. Com esta funcionalidade, você pode acessar seu run ID antes da geração inicial, o que pode ser útil para ações como enviar feedback. O exemplo abaixo demonstra isso.

In [ ]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

In [ ]:
from langsmith import traceable

@traceable
def foo():
    return "Esta é uma Execução de exemplo!"

Estamos passando uma config com nossa chamada de função através de `langsmith_extra` que contém nosso run_id pré-definido

In [ ]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

Agora podemos criar feedback diretamente nesta execução!

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(pre_defined_run_id, "feedback_usuario", score=1)

### URLs de Feedback Pré-assinadas

![presigned url](../../images/presigned_url.png)

Isso também pode ser útil para URLs de feedback pré-assinadas. Você gostaria de usá-las quando não pode expor chaves de API ou outros segredos para o cliente, ex. em uma aplicação web. Usando um run_id pré-determinado, LangSmith tem um endpoint create_presigned_feedback_token que criará uma URL para enviar feedback, sem o uso de segredos necessários.

In [ ]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

In [ ]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "feedback_usuario_pre_assinado")

print(pre_signed_url)

Aqui, podemos ver que mesmo que ainda não tenhamos criado uma execução, ainda somos capazes de gerar a URL de feedback.

Agora, vamos invocar nossa cadeia para que a execução com aquele ID seja criada:

In [ ]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

Então, uma vez que nossa execução é criada, podemos usar a URL de feedback para enviar feedback:

In [ ]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback enviado com sucesso!")
else:
    print("Falha no envio do feedback!")